# Mini GPT: Building a Language Model from Scratch

In this session, we will **build and train a simplified GPT-like model** step by step (similar to Karpathy's `nanoGPT`).  
By the end, you will understand the **core building blocks** behind modern large language models.

### What to expect
- Learn how text is converted into numbers the model can understand.  
- Explore how Transformers work: **embeddings, self-attention, feedforward layers, and residual connections**.  
- Put these pieces together into a small GPT model.  
- Train the model on Shakespeare’s text.  
- Generate new, Shakespeare-like text from scratch.  

⚡ This is a **hands-on educational exercise**: the model is tiny compared to real GPTs, but it captures the **same core ideas**.

---

### Setup and Hyperparameters

We start by importing **PyTorch** (`torch`) and some of its modules:

- `torch.nn`: building blocks for neural networks.
- `torch.nn.functional`: activation functions, loss functions, and other utilities.

Then we define our **hyperparameters** — these control how the model is trained and how large it is:

- `batch_size = 64`: how many training sequences we process at the same time.
- `block_size = 256`: the maximum length of context (how many previous characters the model can "see" when predicting the next one).
- `max_iters = 5000`: number of training steps.
- `eval_interval = 500`: how often we evaluate model performance on validation data.
- `learning_rate = 3e-4`: how fast the optimizer updates weights.
- `device`: whether to use GPU (`cuda`) or CPU.
- `eval_iters = 200`: number of mini-batches used to estimate loss during evaluation.
- `n_embd = 384`: size of the embedding vectors (each token is represented by a vector of this length).
- `n_head = 6`: number of self-attention heads in the Transformer.
- `n_layer = 6`: number of Transformer blocks stacked together.
- `dropout = 0.2`: fraction of neurons randomly “dropped” during training to prevent overfitting.

These values define both the **capacity** of the model and the **training process**.


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2


### Loading and Preparing the Dataset

1. **Random seed**  
   `torch.manual_seed(1337)` ensures reproducibility — we’ll get the same random numbers each run.

2. **Load text**  
   We read Shakespeare’s works (`input.txt`) into a single string called `text`.

3. **Vocabulary**  
   - `chars = sorted(list(set(text)))`: all unique characters that appear in the dataset.  
   - `vocab_size`: total number of unique characters.  
     Example: for Shakespeare it’s usually around 65 (letters, punctuation, space, etc.).

4. **Character ↔ Integer mapping**  
   - `stoi` (“string to integer”): maps each character to an index.  
   - `itos` (“integer to string”): reverse mapping.  
   - `encode(s)`: converts a string into a list of integers.  
   - `decode(l)`: converts a list of integers back into text.

   > This is how we turn raw text into numerical data that a neural network can process.

5. **Dataset split**  
   - Encode the entire text into a `torch.tensor` of integers.  
   - Use the first 90% as `train_data`.  
   - Keep the last 10% as `val_data` (to measure generalization).


In [ ]:
torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

### Creating Training Batches

`get_batch(split)` samples random chunks of text:

- Picks `batch_size` random starting positions.  
- Builds input `x` (current characters) and target `y` (the same sequence shifted by one character).  
- Moves tensors to the right device (CPU/GPU).

Result: `x, y` have shape `(batch_size, block_size)` and are used to train the model to predict the **next character**.


In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

### Estimating Loss

`estimate_loss()` checks how well the model is doing without updating weights:

- Disables gradient tracking (`@torch.no_grad()` → faster, less memory).  
- Switches model to evaluation mode (`model.eval()` disables dropout).  
- Runs `eval_iters` batches for both **train** and **val** sets.  
- Collects and averages the losses.  
- Switches back to training mode (`model.train()`).  

This gives a stable estimate of training vs validation performance.


In [ ]:
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

### Self-Attention Head

A single head of self-attention:

- Projects input `x` into **queries**, **keys**, and **values**.  
- Computes attention scores (`q @ k^T`), scaled for stability.  
- Applies a **causal mask** (`tril`) so tokens can’t look ahead.  
- Softmax → probabilities over past positions.  
- Uses these to weight the values → output.

Input shape: `(batch, time, channels)`  
Output shape: `(batch, time, head_size)`


In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

### Multi-Head Attention

Instead of one head, we use several in parallel:

- Each `Head` learns different attention patterns.  
- Their outputs are concatenated (`torch.cat`) along the channel dimension.  
- A final linear projection mixes them back into the embedding size (`n_embd`).  
- Dropout is applied for regularization.

This allows the model to attend to different types of relationships at once.

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

### FeedForward Layer

A position-wise MLP applied to each token:

- Expands embedding size (`n_embd → 4*n_embd`).  
- Applies ReLU non-linearity.  
- Projects back down to `n_embd`.  
- Adds dropout for regularization.

This gives the model extra capacity beyond attention.


In [ ]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

### Transformer Block

Each block combines **communication (attention)** and **computation (MLP)**:

1. Apply LayerNorm → Multi-Head Attention → add residual connection.  
2. Apply LayerNorm → FeedForward → add residual connection.  

This structure lets tokens share information while keeping stable gradients.  
Stacking several blocks builds the full Transformer.


In [ ]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

### GPTLanguageModel

This class assembles the full Transformer-based language model:

- **Token embeddings**: turn character indices into vectors.  
- **Position embeddings**: add information about order in the sequence.  
- **Stack of Transformer blocks**: attention + feedforward layers.  
- **Final LayerNorm + Linear head**: map hidden states to vocabulary logits.  

**Forward pass**:
1. Look up token + position embeddings → combine them.  
2. Pass through Transformer blocks.  
3. Project to logits over the vocabulary.  
4. If targets are provided, compute cross-entropy loss.  

Output: `(logits, loss)` where  
- `logits`: predictions for next token.  
- `loss`: training signal (or `None` if not given).


In [ ]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

### GPTLanguageModel

This class assembles the full Transformer-based language model:

- **Token embeddings**: turn character indices into vectors.  
- **Position embeddings**: add information about order in the sequence.  
- **Stack of Transformer blocks**: attention + feedforward layers.  
- **Final LayerNorm + Linear head**: map hidden states to vocabulary logits.  

**Forward pass**:
1. Look up token + position embeddings → combine them.  
2. Pass through Transformer blocks.  
3. Project to logits over the vocabulary.  
4. If targets are provided, compute cross-entropy loss.  

Output: `(logits, loss)` where  
- `logits`: predictions for next token.  
- `loss`: training signal (or `None` if not given).


In [ ]:
def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
        # crop idx to the last block_size tokens
        idx_cond = idx[:, -block_size:]
        # get the predictions
        logits, loss = self(idx_cond)
        # focus only on the last time step
        logits = logits[:, -1, :] # becomes (B, C)
        # apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1) # (B, C)
        # sample from the distribution
        idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
        # append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

### Model Initialization and Optimizer

- Create the model (`GPTLanguageModel`) and move it to the right device (CPU/GPU).  
- Print the total number of parameters (in millions) → shows model size.  
- Define the optimizer: **AdamW**, a variant of Adam with weight decay, commonly used for Transformers.


In [ ]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

### Training Loop

For `max_iters` steps:

1. **Evaluate periodically**: every `eval_interval` steps (or at the end), estimate train/val loss.  
2. **Get a batch**: `(xb, yb)` input and target sequences.  
3. **Forward pass**: compute `logits` and `loss`.  
4. **Backward pass**:  
   - Reset gradients (`optimizer.zero_grad`).  
   - Backpropagate (`loss.backward()`).  
   - Update weights (`optimizer.step()`).  

This loop gradually teaches the model to predict the next character.


In [ ]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

### Text Generation

- Start with a single token (`0`) as the initial context.  
- Call `model.generate(...)` to autoregressively sample the next tokens, up to `max_new_tokens=500`.  
- Decode the generated token IDs back into text and print it.  

👉 The model produces new Shakespeare-like text, character by character.


In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

---

## How This Mini GPT Differs from Real GPT Models

Our model is inspired by GPT, but it’s much smaller and simpler.  
Here are five key differences:

1. **Scale**  
   - GPT (e.g., GPT-3) has **billions of parameters**, trained on huge datasets with thousands of GPUs.  
   - Our mini GPT has only a few **million parameters**, trained on a tiny dataset (Shakespeare) with a single GPU/CPU.

2. **Tokenizer**  
   - GPT uses **Byte Pair Encoding (BPE)** or similar subword tokenization → efficient and works for all languages.  
   - Our version uses **character-level tokens** → simpler, but less efficient for large vocabularies.

3. **Architecture Details**  
   - GPT includes improvements like **pre-layer normalization**, **rotary embeddings**, and optimized attention implementations (e.g. FlashAttention).  
   - Our mini GPT uses a **basic Transformer** with embeddings, attention, and feedforward layers.

4. **Training Setup**  
   - GPT is trained with **massive compute budgets** for weeks or months.  
   - Our model trains in **hours** (or less) on a laptop GPU.

5. **Purpose**  
   - GPT is built for **general-purpose applications** (chat, coding, search, etc.).  
   - Our mini GPT is designed for **learning and experimentation** — to show the core ideas in action.

# MCQ: Tuesday with the slides